In [13]:
# fold
import cv2
import glob
import matplotlib.pyplot as plt
import math
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import optimizers
# For some reason I have to tell it to use TensorFlows dimension ordering
from keras import backend as K
K.set_image_dim_ordering('tf')
from keras.callbacks import TensorBoard
from time import time

def get_images(path, dataSpecs):
    fileList = glob.glob(path) #'BengaliBMPConvert/*.bmp'   
    num = len(fileList)
    dataSpecs['classLength'].append(len(fileList))
    x = np.array([(cv2.imread(fname)) for fname in fileList])
    return x

def get_labels_one_hot(num_classes, class_id, num_samples):
    x = np.zeros((num_samples, num_classes))
    x[np.arange(num_samples),class_id] = 1
    return x
 
    
    
def main():
    import keras
    print('keras version: ', keras.__version__)
    
    # Get images and labels
    data = {'img': 0, 'label': 0}
    dataSpecs = {'classLength': []}
    dataSpecs['classLength'] = []
    data['img'] = np.concatenate((
            #get_images('/vol/vssp/mammo2/will/data/simulated/calcs/small_sample/0/*', dataSpecs), # Class 0
            #get_images('/vol/vssp/mammo2/will/data/simulated/calcs/small_sample/1/*', dataSpecs) # Class 1
            get_images('/user/HS204/wm0015/student/allCalcs/0/*', dataSpecs), # Class 0
            get_images('/user/HS204/wm0015/student/allCalcs/1/*', dataSpecs) # Class 1
    ))  
    # Normalise
    data['img'] = data['img']/255
    
    print('dataSpecs = ', dataSpecs['classLength'])
    labels_bg = get_labels_one_hot(2, 0, dataSpecs['classLength'][0])  
    labels_calc = get_labels_one_hot(2, 1, dataSpecs['classLength'][1])
    data['label'] = np.concatenate((
            get_labels_one_hot(2, 0, dataSpecs['classLength'][0]), # Class 0 
            get_labels_one_hot(2, 1, dataSpecs['classLength'][1]) # Class 1
    ))
    # Drop from 3 colour channels to 1 (greyscale)
    if 1==1:
        data['img'] = data['img'][:,:,:,0]
        data['img'] = np.reshape(data['img'], (data['img'].shape[0],data['img'].shape[1],data['img'].shape[2],1))
        print('new data shape = ', data['img'].shape)
    
    
    
    
    # Shuffle data
    seed = 33
    np.random.seed(seed) # Has to be set before each use of random
    shuffleMask = np.random.permutation(data['img'].shape[0])    
    data['img'] = data['img'][shuffleMask, :, :, :]
    data['label'] = data['label'][shuffleMask, :]
    
    # Split traing and validation data        
    splitRatio = 0.9
    splitPoint = math.floor(data['img'].shape[0]*splitRatio)
    train_data = {'img': data['img'][0:splitPoint], 'label': data['label'][0:splitPoint]}
    validation_data = {'img': data['img'][splitPoint:], 'label': data['label'][splitPoint:]}
    print('train_data[img].shape = ', train_data['img'].shape)
    print('train_data[label].shape = ', train_data['label'].shape)
    print('validation_data[img].shape = ', validation_data['img'].shape)
    print('valdiation_data[label].shape = ', validation_data['label'].shape)
    
    # Print image    
#     img_calc = train_data['img']   
#     plt.imshow(img_calc[0], cmap='gray')
#     plt.show()
    
    # Keras!
    model = Sequential()
    
    # Layer 1
    model.add(Conv2D(32, (3,3), activation='relu', input_shape=(385, 385, 1), data_format='channels_last'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 2
    model.add(Conv2D(32, (3,3), activation='relu')) 
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 3    
    model.add(Conv2D(32, (3,3), activation='relu'))  
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 4     
    model.add(Conv2D(32, (3,3), activation='relu'))   
    model.add(MaxPooling2D(pool_size=(2, 2)))    
    # Layer 5     
    model.add(Conv2D(32, (3,3), activation='relu'))   
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 6
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    
    sgd = optimizers.SGD(lr=5, decay=1e-6, momentum=0.9, nesterov=True)
    adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False) #0.001
    model.compile(loss='binary_crossentropy',
            optimizer=adam,
            metrics=['accuracy']) # Accuracy will now be returned when evaluate is called
    tensorboard = TensorBoard(log_dir='/vol/vssp/mammo2/will/logs/new'.format(time()))
    model.summary()   
    
    # Train
    model.fit(train_data['img'], train_data['label'], 
            batch_size=100, epochs=100, verbose=1,
            callbacks=[tensorboard])
    
    # Evaluate
    score = model.evaluate(validation_data['img'], validation_data['label'], verbose=0)
    print('The score is......\n', score)
    
if __name__ == "__main__":
    main()

keras version:  2.2.0
dataSpecs =  [901, 719]
new data shape =  (1620, 385, 385, 1)
train_data[img].shape =  (1458, 385, 385, 1)
train_data[label].shape =  (1458, 2)
validation_data[img].shape =  (162, 385, 385, 1)
valdiation_data[label].shape =  (162, 2)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_34 (Conv2D)           (None, 383, 383, 32)      320       
_________________________________________________________________
max_pooling2d_34 (MaxPooling (None, 191, 191, 32)      0         
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 189, 189, 32)      9248      
_________________________________________________________________
max_pooling2d_35 (MaxPooling (None, 94, 94, 32)        0         
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 92, 92, 32)        9248      
__________________

1458/1458 [==============================] - 5s 3ms/step - loss: 8.8013e-05 - acc: 1.0000
Epoch 59/100
1458/1458 [==============================] - 5s 4ms/step - loss: 2.8577e-05 - acc: 1.0000
Epoch 60/100
1458/1458 [==============================] - 5s 3ms/step - loss: 2.7079e-05 - acc: 1.0000
Epoch 61/100
1458/1458 [==============================] - 5s 3ms/step - loss: 4.1263e-05 - acc: 1.0000
Epoch 62/100
1458/1458 [==============================] - 5s 4ms/step - loss: 2.0949e-05 - acc: 1.0000
Epoch 63/100
1458/1458 [==============================] - 5s 3ms/step - loss: 2.5922e-05 - acc: 1.0000
Epoch 64/100
1458/1458 [==============================] - 5s 4ms/step - loss: 7.3268e-05 - acc: 1.0000
Epoch 65/100
1458/1458 [==============================] - 4s 3ms/step - loss: 2.2889e-05 - acc: 1.0000
Epoch 66/100
1458/1458 [==============================] - 5s 3ms/step - loss: 4.5048e-05 - acc: 1.0000
Epoch 67/100
1458/1458 [==============================] - 5s 3ms/step - loss: 8.4739e-

In [ ]:
print('hello world')